# Installing Python Packages from a Jupyter Notebook

<!--PELICAN_BEGIN_SUMMARY-->

In software, it's said that [all abstractions are leaky](https://www.joelonsoftware.com/2002/11/11/the-law-of-leaky-abstractions/), and this is true for the Jupyter notebook as it is for any other software.
I most often see this manifest itself with the following issue:

> I installed *package X* and now I can't import it in the notebook. Help!
    
This issue is a perrennial source of StackOverflow questions (e.g. [this](https://stackoverflow.com/questions/39007571/running-jupyter-with-multiple-python-and-ipython-paths/), [that](https://stackoverflow.com/questions/42500142/importerror-no-module-named-jwt-in-jupyter), [here](https://stackoverflow.com/questions/32777807/importerror-no-module-named-cv2-using-jupyter), [there](https://stackoverflow.com/questions/42500649/failed-to-import-numpy-as-np-when-i-worked-with-jupyter-notebook), [another](https://stackoverflow.com/questions/46634660/jupyter-notebook-wrong-sys-path-and-sys-executable), [this one](https://stackoverflow.com/questions/44222513/cannot-import-datashader-installed-using-miniconda), [that one](https://stackoverflow.com/questions/42178070/jupyter-notebook-importerror-no-module-named-sklearn), [and this](https://stackoverflow.com/questions/42034508/fail-pandas-in-python3-jupyter-notebook)... etc.).
Fundamentally the problem usually comes down to the fact that the Jupyter notebook is pointing to a different Python kernel than the package manager the user invokes.
In the simplest use-cases this issue does not arise, but when it does, debugging the problem requires knowledge of both the intricacies of the operating system, **and** the intricacies of Jupyter itself.
In other words, the Jupyter notebook, like all abstractions, is leaky.

The leakiness of Jupyter's abstraction doesn't mean it's bad — all abstractions are leaky, after all — but it does mean that developers should think deeply about how to better help users who run into the leaks.

In the wake of several discussions on this topic with colleagues, both online ([exhibit A](https://twitter.com/amuellerml/status/932637063748444160), [exhibit B](https://twitter.com/jakevdp/status/922846245848150016)) and off, I decided to tackle the issue here.
I'll address a couple things:

- **First**, I'll provide a quick, bare-bones answer to the general question, *how can I install a Python package so it works with my jupyter notebook, using pip and/or conda?*.

- **Second**, I'll dive into some of the background of exactly *what* the Jupyter notebook abstraction is doing, how it interacts with the complexities of the operating system, and how you can think about where the "leaks" are, and thus better understand and debug the 1% of times that things stop working.

- **Third**, I'll talk about some ideas the community might consider to help smooth-over these issues, including some changes that the Jupyter, Pip, and Conda tools might consider to ease the cognitive load on users.

This post will focus on two approaches to installing Python packages: [pip](https://pip.pypa.io/en/stable/) and [conda](https://conda.io/docs/).
Other package managers exist (including platform-specific tools like [yum](http://yum.baseurl.org/), [apt](https://help.ubuntu.com/community/AptGet/Howto#Package_management_with_APT), [homebrew](https://brew.sh/), etc., as well as cross-platform tools like [enstaller](http://enstaller.readthedocs.io/en/latest/)), but I'm less familiar with them and won't be remarking on them further.

<!--PELICAN_END_SUMMARY-->

## Quick Fix: How To Install Packages from the Jupyter Notebook

If you're just looking for a quick answer to the question, *how do I install packages so they work with the notebook*, then look no further.

### pip vs. conda

First, a few words on ``pip`` vs. ``conda``.
For many users, the choice between pip and conda can be a confusing one.
I wrote [way more than you ever want to know](https://jakevdp.github.io/blog/2016/08/25/conda-myths-and-misconceptions/) about these in a post last year, but the essential difference between the two is this:

- *pip* installs **python** packages in **any environment**.
- *conda* installs **any** package in **conda environments**.

If you already have a Python installation that you're using, then the choice of which to use is easy:

- If you installed Python using Anaconda or Miniconda, then use ``conda`` to install Python packages. If conda tells you the package you want doesn't exist, then use pip.
  
- If you installed Python any other way (from source, using pyenv, virtualenv, etc.), then use ``pip`` to install Python packages

That's all there is to it!

### How to use Conda from the Jupyter Notebook

If you're in the jupyter notebook and you want to install a package with conda, you might be tempted to use the ``!`` notation to run conda directly as a shell command from the notebook:

In [1]:
# DON'T DO THIS!
!conda install --yes numpy

Fetching package metadata ...........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/jakevdp/anaconda/envs/python3.6:
#
numpy                     1.13.3           py36h2cdce51_0  


(Note that we use ``--yes`` to automatically answer ``y`` if and when conda asks for user confirmation)

For various reasons that I'll outline more fully below, this **will not generally work** if you want to use these installed packages from Jupyter, though it may work in the simplest cases.

Here is a short snippet that should work in general:

In [2]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} numpy

Fetching package metadata ...........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/jakevdp/anaconda:
#
numpy                     1.13.3           py36h2cdce51_0  


That bit of extra boiler-plate makes certain that conda installs the package in the currently-running Jupyter kernel (thanks to [Min Ragan-Kelley](https://twitter.com/minrk/status/842067777150169088) for suggesting this approach).
I'll discuss why this is needed momentarily.

### How to use Pip from the Jupyter Notebook

If you're using the Jupyter notebook and want to install a package with ``pip``, you similarly might be inclined to run pip directly in the shell:

In [3]:
# DON'T DO THIS
!pip install numpy

For various reasons that I'll outline more fully below, this **will not generally work** if you want to use these installed packages from Jupyter, though it may work in the simplest cases.

Here is a short snippet that should generally work:

In [4]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy

That bit of extra boiler-plate makes certain that you are running the ``pip`` version associated with the current Python kernel, so that the installed packages can be used in the current notebook.
This is related to the fact that, even setting Jupyter notebooks aside, it's better to install packages using

    $ python -m pip install <package>
    
rather than

    $ pip install <package>
    
because the former is more explicit about where the package will be installed.

``python -m pip install`` is the only form recommended in the [current Python docs](https://docs.python.org/3/installing/index.html#basic-usage), and there are even [indications](https://twitter.com/ncoghlan_dev/status/922979220711661568) that the less-explicit ``pip install`` may someday be deprecated.

I'll go into more detail on the problems with ``pip install`` shortly.

## The Details: Why is Installation from Jupyter so Messy?

Those above solutions should work in all cases... but why is that additional boilerplate necessary?
In short, it's because in Jupyter, **the shell environment and the Python environment are disconnected**.
Understanding why that matters depends on a basic understanding of a few different concepts:

1. how your operating system locates executable programs,
2. how Python installs and locates packages
3. how Jupyter decides which Python executable to use.

For completeness, I'm going to delve briefly into each of these topics (this discussion is partly drawn from [This StackOverflow answer](https://stackoverflow.com/questions/39007571/running-jupyter-with-multiple-python-and-ipython-paths/39022003#39022003) that I wrote last year).

*Note: the following discussion assumes Linux, Unix, MacOSX and similar operating systems. Windows has a slightly different architecture, and so some details will differ.*

### How your operating system locates executables

When you're using the terminal and type a command like ``python``, ``jupyter``, ``ipython``, ``pip``, ``conda``, etc., your operating system contains a well-defined mechanism to find the executable file the name refers to.
On Linux & Mac systems, this mechanism is the ``$PATH`` environment variable:

In [5]:
!echo $PATH

/Users/jakevdp/anaconda/envs/python3.6/bin:/Users/jakevdp/anaconda/envs/python3.6/bin:/Users/jakevdp/anaconda/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin


``$PATH`` lists the directories, in order, that will be searched for any executable: for example, if I type ``python`` on my system with the above ``$PATH``, it will first look for ``/Users/jakevdp/anaconda/envs/python3.6/bin/python``, and if that doesn't exist it will look for ``/Users/jakevdp/anaconda/bin/python``, and so on.

(Parenthetical note: why is the first entry of ``$PATH`` repeated twice? Because every time you launch ``jupyter notebook``, Jupyter prepends the location of the ``jupyter`` executable to the beginning of the ``$PATH``. In this case, the location was already at the beginning of the path, and the result is that the entry is duplicated. Duplicate entries add clutter, but cause no harm).

You can use the ``which`` command to see beforehand which ``python`` executable is the first found:

In [6]:
!which python

/Users/jakevdp/anaconda/envs/python3.6/bin/python


Note that this is true of *any* command you use from the terminal:

In [7]:
!which ls

/bin/ls


even ``which`` itself:

In [8]:
!which which

/usr/bin/which


You can optionally add the ``-a`` tag to see *all available* executables of that name in your path; for example:

In [9]:
!which -a python

/Users/jakevdp/anaconda/envs/python3.6/bin/python
/Users/jakevdp/anaconda/envs/python3.6/bin/python
/Users/jakevdp/anaconda/bin/python
/usr/bin/python


In [10]:
!which -a conda

/Users/jakevdp/anaconda/envs/python3.6/bin/conda
/Users/jakevdp/anaconda/envs/python3.6/bin/conda
/Users/jakevdp/anaconda/bin/conda


In [11]:
!which -a pip

/Users/jakevdp/anaconda/envs/python3.6/bin/pip
/Users/jakevdp/anaconda/envs/python3.6/bin/pip
/Users/jakevdp/anaconda/bin/pip


When you have multiple versions of any executable, it is important to keep in mind the role of ``$PATH`` in choosing which will be used.

### How Python locates packages

Python uses a similar mechanism to locate imported packages.
The list of paths searched by Python on import is found in ``sys.path``:

In [12]:
import sys
sys.path

['',
 '/Users/jakevdp/anaconda/lib/python36.zip',
 '/Users/jakevdp/anaconda/lib/python3.6',
 '/Users/jakevdp/anaconda/lib/python3.6/lib-dynload',
 '/Users/jakevdp/anaconda/lib/python3.6/site-packages',
 '/Users/jakevdp/anaconda/lib/python3.6/site-packages/schemapi-0.3.0.dev0+791c7f6-py3.6.egg',
 '/Users/jakevdp/anaconda/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg',
 '/Users/jakevdp/anaconda/lib/python3.6/site-packages/IPython/extensions',
 '/Users/jakevdp/.ipython']

By default, the first place Python looks for a module is an empty path, meaning the current directory.
If the module is not found there, it goes down the list of locations until the module is found.
You can find out which location has been used using the ``__path__`` attribute of the module:

In [13]:
import numpy
numpy.__path__

['/Users/jakevdp/anaconda/lib/python3.6/site-packages/numpy']

In most cases, a Python package you install with ``pip`` or with ``conda`` will be put in a directory called ``site-packages``.
The important thing to realize is that each Python executable has **it's own site-packages**: what this means is that when you install a package, it is **associated with particular python executable** and by default can only be used with that Python!

We can see this by printing the ``sys.path`` variables for each of the available ``python`` executables in my path, using Jupyter's delightful ability to mix Python and bash commands in a single code block:

In [14]:
paths = !which -a python
for path in set(paths):
    print('--------')
    print(path)
    !{path} -c "import sys; print(sys.path)"

--------
/Users/jakevdp/anaconda/envs/python3.6/bin/python
['', '/Users/jakevdp/anaconda/envs/python3.6/lib/python36.zip', '/Users/jakevdp/anaconda/envs/python3.6/lib/python3.6', '/Users/jakevdp/anaconda/envs/python3.6/lib/python3.6/lib-dynload', '/Users/jakevdp/anaconda/envs/python3.6/lib/python3.6/site-packages']
--------
/Users/jakevdp/anaconda/bin/python
['', '/Users/jakevdp/anaconda/lib/python36.zip', '/Users/jakevdp/anaconda/lib/python3.6', '/Users/jakevdp/anaconda/lib/python3.6/lib-dynload', '/Users/jakevdp/anaconda/lib/python3.6/site-packages', '/Users/jakevdp/anaconda/lib/python3.6/site-packages/schemapi-0.3.0.dev0+791c7f6-py3.6.egg', '/Users/jakevdp/anaconda/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg']
--------
/usr/bin/python
['', '/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python27.zip', '/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7', '/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/plat-darwin'

The details here are not particularly important, but it is important to emphasize that *each Python executable has its own set of packages*, and unless you change ``sys.path`` (which should only be done with great care) you cannot import packages installed in a different Python environment.

When you run ``pip install`` or ``conda install``, these commands are associated with a single Python version, generally the version that is within the same path:

In [15]:
!which python

/Users/jakevdp/anaconda/envs/python3.6/bin/python


In [16]:
!which pip

/Users/jakevdp/anaconda/envs/python3.6/bin/pip


In [17]:
!which conda

/Users/jakevdp/anaconda/envs/python3.6/bin/conda


When running these commands in the notebook, they go first to ``/Users/jakevdp/anaconda/envs/python3.6/bin/``, because *this is the path from which I started the notebook* with ``jupyter notebook``.
So ``!pip install`` and ``!conda install`` will install packages for ``/Users/jakevdp/anaconda/envs/python3.6/bin/python``, the Python version used to launch the Jupyter notebook.

#### Jupyter's python vs. Jupyter's shell

Now we come to the root of the installation headaches: because of the way Jupyter is architected, the ``python`` at the top of your ``$PATH`` is not necessarily the ``python`` being used by the notebook to execute code.
Within a Python session, we can use ``sys.executable`` to figure out which Python is being used to execute code in the notebook:

In [18]:
sys.executable

'/Users/jakevdp/anaconda/bin/python'

Compare this to

In [19]:
!which python

/Users/jakevdp/anaconda/envs/python3.6/bin/python


The two differ, and this is why a simple ``!pip install`` or ``!conda install`` does not work: the commands install packages in the ``site-packages`` of the wrong Python installation.

As noted above, we can get around this by explicitly identifying where we want packages to be installed.

For **conda**, you can use:

```
$ conda install --yes --prefix /Users/jakevdp/anaconda numpy
```

or more generally (using syntax available in the notebook or IPython terminal)

```
!conda install --yes --prefix {sys.prefix} numpy
```

For **pip**, you can use:

```
$ /Users/jakevdp/anaconda/bin/python -m pip install numpy
```

or more generally (again using IPython shell syntax)

```
!{sys.executable} -m pip install numpy
```

Remember: you need your *installation command* to match the *current python executable* if you want installed packages to be available in the notebook.

### How Jupyter executes code: Jupyter Kernels

So what is the reason the shell and the Python executable don't necessarily match?
This goes to the question of how Jupyter decides how it will execute code, and this brings us to the concept of a *Jupyter Kernel*.

A Jupyter kernel is a set of files that point Jupyter to some means of executing code within the notebook.
For Python kernels, this will point to a particular Python version, but Jupyter is designed to be much more general than this: Jupyter has [dozens of available kernels](https://github.com/jupyter/jupyter/wiki/Jupyter-kernels) for languages including Julia, Python, R, Ruby, Haskell, and even C++ and Fortran!

To see the kernels you have available on your system, you can run the following command in the shell:

In [20]:
!jupyter kernelspec list

Available kernels:
  python3       /Users/jakevdp/anaconda/envs/python3.6/lib/python3.6/site-packages/ipykernel/resources
  conda-root    /Users/jakevdp/Library/Jupyter/kernels/conda-root
  myenv         /Users/jakevdp/Library/Jupyter/kernels/myenv
  python2.7     /Users/jakevdp/Library/Jupyter/kernels/python2.7
  python3.5     /Users/jakevdp/Library/Jupyter/kernels/python3.5
  python3.6     /Users/jakevdp/Library/Jupyter/kernels/python3.6


Each of these listed kernels is a directory that contains a file called ``kernel.json`` which specifies, among other things, which language and executable the kernel should use.
For example:

In [21]:
!cat /Users/jakevdp/Library/Jupyter/kernels/conda-root/kernel.json

{
 "argv": [
  "/Users/jakevdp/anaconda/bin/python",
  "-m",
  "ipykernel_launcher",
  "-f",
  "{connection_file}"
 ],
 "display_name": "python (conda-root)",
 "language": "python"
}


If you'd like to create a new kernel, you can do so using the [jupyter ipykernel command](http://ipython.readthedocs.io/en/stable/install/kernel_install.html#kernels-for-different-environments);
for example, I created the above kernels for my primary conda environments using the following as a template:
```
$ source activate myenv
$ python -m ipykernel install --user --name myenv --display-name "Python (myenv)"
```
If you wish, there are also more automated ways to link Jupyter kernels to conda environments; see for example the [nb_conda package](https://github.com/Anaconda-Platform/nb_conda).

The mismatch between shell paths and executable paths happens because the Python kernel specification does not modify the shell environment of the notebook to match the Python envorinment.
Because of this ``!pip install`` and ``!conda install`` do not always install packages where you might expect them to.

## Some Modest Proposals

So, in summary, the reason that installation of packages in the Jupyter notebook is fraught with difficulty is fundamentally that **Jupyter's shell environment and Python kernel are mismatched**, and that means that you have to do more than simply ``pip install`` or ``conda install`` to make things work.
The exception is the special case where you run ``jupyter notebook`` from the same Python environment to which your kernel points; in that case the simple installation approach should work.

But that leaves us in an undesireable place, as it increases the learning curve for novice users who may want to do something they (rightly) presume should be simple: install a package and then use it.
So what can we as a community do to smooth-out this issue?

I have a few ideas, some of which might even be useful.

### Potential Changes to Jupyter

As I mentioned, the fundamental issue is a mismatch between Jupyter's shell environement and compute kernel.
So, could we massage kernel specifications such that they force the shell environment to match the Python kernel?

Perhaps: for example, [this github issue](https://github.com/jupyterhub/jupyterhub/issues/847)) shows an example of initializing shell variables as part of a kernel.

Basically, in your kernel directory, you can add a script ``kernel-startup.sh`` that looks something like this (and make sure you change the permissions so that it's executable):

```
#!/usr/bin/env bash

# activate anaconda env
source activate myenv

# this is the critical part, and should be at the end of your script:
exec python -m ipykernel $@
```

Then in your ``kernel.json`` file, modify the ``argv`` field to look like this:

```
"argv": [
   "/path/to/kernel-startup.sh",
   "-f",
   "{connection_file}"
 ]
```

Once you do this, switching to the ``myenv`` kernel will automatically activate the ``myenv`` conda environment, which changes your ``$PATH`` and other system variables such that ``!conda install XXX`` and ``!pip install XXX`` will work correctly. A similar approach could work for virtualenvs.

There is one tricky issue here: this approach will fail if your ``myenv`` environment does not have the ``jupyter`` package installed, and probably requires it to have a jupyter version compatible with that used to launch the notebook originally. So it's not a full solution to the problem, but this may point to a path forward that would be far less confusing for users, if Python kernels could be set-up to do this sort of thing by default.

### Potential Changes to pip

One source of installation confusion, even outside of Jupyter, is the fact that, depending on the nature of your system's ``$PATH`` variable, ``pip`` and ``python`` might point to different paths.
In this case ``pip install`` will install packages to a path inaccessible to the ``python`` executable.
This is one reason that ``pip install`` no longer appears in Python's docs.
CPython developer Nick Coghlan has even [indicated](https://twitter.com/ncoghlan_dev/status/922979220711661568) that ``pip`` may someday be deprecated in favor of ``python -m pip``.
Though it's more verbose, I think this would be a useful change from the point of view of users, particularly as the use of virtualenvs and conda envs becomes more common.

### Changes to Conda

For symmetry, it would be nice if ``python -m conda install`` could be expected to work in the same way the ``pip`` counterpart does.
You can call ``conda`` this way in the root environment, but the conda Python package (as opposed to the conda executable) cannot currently be installed anywhere but the root environment; if you try you get the following:
```
(myenv) jakevdp$ conda install conda
Fetching package metadata ...........

InstallError: Error: 'conda' can only be installed into the root environment
```
I suspect that allowing ``python -m conda install`` in all conda environments would require a fairly significant redesign of conda's installation model, so it may not be worth the change just for symmetry with ``pip``.

### New Jupyter Magic Functions

Even if the above changes to the stack are not possible or desirable, we could simplify the user experience somewhat by introducing ``%pip`` and ``%conda`` magic functions within the Jupyter notebook that detect the current kernel and make certain packages are installed in the correct location.

#### pip magic

For example, here's how you can define a ``%pip`` magic function that works in the current kernel:

In [22]:
from IPython.core.magic import register_line_magic

@register_line_magic
def pip(args):
    """Use pip from the current kernel"""
    from pip import main
    main(args.split())

Running it as follows will install packages in the expected location

In [23]:
%pip install numpy

Note that Jupyter developer Matthias Bussonnier has published essentially this in his [pip_magic](https://github.com/Carreau/pip_magic) repository, so you can do
```
$ pip install pip_magic
```
and use this right now (that is, assuming you install ``pip_magic`` in the right place!)

#### conda magic

Similarly, we can define a conda magic that will do the right thing if you type ``%conda install XXX``.
This is a bit more complicated than the ``pip`` magic, because it must first confirm that the environment is conda-compatible, and then call a subprocess to execute the appropriate shell command:

In [24]:
from IPython.core.magic import register_line_magic
import sys
import os
from subprocess import Popen, PIPE
from unittest.mock import patch


@register_line_magic
def conda(args):
    """Use conda from the current kernel"""        
    # First, check if this Python installation uses conda. All conda envs have
    # a conda executable in the same path as the python executable
    # and a 'conda-meta/history' file in the prefix
    conda_path = os.path.join(os.path.dirname(sys.executable), 'conda')  # OSX/Linux
    conda_exe_path = os.path.join(os.path.dirname(sys.executable), 'conda.exe')  # Windows
    
    conda_executable_exists = os.path.exists(conda_path) or os.path.exists(conda_exe_path)
    conda_meta_exists = os.path.exists(os.path.join(sys.prefix, 'conda-meta', 'history'))
    
    if not (conda_meta_exists and conda_executable_exists):
        raise ValueError("The python kernel does not appear to be a conda environment.  "
                         "Please use ``%pip install`` instead.")
    
    # Add --prefix to point conda installation to the current environment
    # Additionally, because the notebook does not allow us to respond "yes" during the
    # installation, we need to insert a --yes in the arguments to conda.
    args = args.split()
    if args[0] in ['install', 'update', 'upgrade', 'remove', 'uninstall', 'create']:
        if '-p' not in args and '--prefix' not in args:
            args.insert(1, '--prefix')
            args.insert(2, sys.prefix)
        if '-y' not in args and '--yes' not in args:
            args.insert(1, '--yes')
            
    # Call conda from command line with subprocess & send results to stdout & stderr
    # TODO: fix conda_path to work with Windows
    # TODO: fix string encoding to work with Python 2 (or not...)
    with Popen([conda_path] + args, stdout=PIPE, stderr=PIPE) as process:
        # Read stdout character by character, as it includes real-time progress updates
        for c in iter(lambda: process.stdout.read(1), b''):
            sys.stdout.write(c.decode(sys.stdout.encoding))
        # Read stderr line by line, because real-time does not matter
        for line in iter(process.stderr.readline, b''):
            sys.stderr.write(line.decode(sys.stderr.encoding))

You can now use ``%conda install`` and it will install packages to the correct environment:

In [25]:
%conda install pandas

Fetching package metadata ...........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/jakevdp/anaconda:
#
pandas                    0.21.0           py36hfed917e_1  


This conda magic still needs some work to be a general solution, but this would be a useful start.

If a pip magic and conda magic similar to the above were added to Jupyter, I think it could go a long way toward solving the common problems that users have when trying to install Python packages for use with Jupyter notebooks.
This approach is not without its own dangers, though: these magics are yet another layer of abstraction that, like all abstractions, will inevitably leak.
But if they are implemented carefully, I think it would lead to a much nicer overall experience for Python users in Jupyter.

## Summary

In this post, I tried to answer once and for all the perrennial question, "how do I install Python packages in the Jupyter notebook".
After proposing some simple solutions that can be used today, I went into a detailed explanation of *why* these solutions are necessary: it comes down to the fact that in Jupyter, the shell and the kernel are entirely disconnected.
The fact that a full explanation took so many words and touched so many concepts, I think, indicates a real usability issue for the Jupyter ecosystem.
Finally, I proposed a few possible avenues that the community might adopt to try to streamline this process.
I hope that these ideas are useful.

One final addendum: I have a huge amount of respect and appreciation for the developers of Jupyter, Conda, Pip, and related tools that form the foundations of the Python data science ecosystem.
I'm fairly certain those developers have already considered these issues and weighed some of these potential fixes – if any of you are reading this, please feel free to comment and set me straight on anything I've overlooked!
And, finally, thanks for all that you do.